In [1]:
# agent benefit
# 1. modularity
# 2. specialization
# 3. control

In [2]:
# multi agent architecture
# 1. single agent 
# 2. network
# 3. supervisor
# 4. supervisor (as tool)
# 5. hierarchical 
# 6. custom 

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [16]:
# handoff - one agent give control to another agent, (destination, payload)
from langgraph.graph import StateGraph, MessagesState
from langgraph.constants import START, END
from langgraph.types import interrupt, Command


def agent_2(state: MessagesState) -> MessagesState:
    print(f"Call agent-2: {state["messages"][-1].content}")
    return {"messages": state["messages"]}

def agent_1(state: MessagesState) -> MessagesState:
    print("call agent 1")
    return Command(
        goto="agent_2",
        update={"messages": "called by agent_1 to agent_2"}
    )

workflow = (
    StateGraph(MessagesState)
    .add_node("agent_1", agent_1)
    .add_node("agent_2", agent_2)
    .set_entry_point("agent_1")
    .compile()
)


In [17]:
workflow.invoke({"messages":"hello"})

call agent 1
Call agent-2: called by agent_1 to agent_2


{'messages': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}, id='ef0236ee-dc3b-4639-a68e-acd57b7ce5cf'),
  HumanMessage(content='called by agent_1 to agent_2', additional_kwargs={}, response_metadata={}, id='d77a49fa-b6b6-424c-9375-d60c9caa2829')]}

#### Handoff as Tool

In [18]:
from langchain_core.tools import tool

@tool
def transfer_to_bob():
    """Transfer to bob."""
    return Command(
        # name of the agent (node) to go to
        goto="bob",
        # data to send to the agent
        update={"my_state_key": "my_state_value"},
        # indicate to LangGraph that we need to navigate to
        # agent node in a parent graph
        graph=Command.PARENT,
    )

#### Network

In [95]:
from typing import Literal
from langgraph.types import Command
from langgraph.graph import StateGraph, MessagesState, START, END
import random
from langgraph.constants import START, END


def agent_1(state: MessagesState) -> Command[Literal["agent_2", END]]:
    # you can pass relevant parts of the state to the LLM (e.g., state["messages"])
    # to determine which agent to call next. a common pattern is to call the model
    # with a structured output (e.g. force it to return an output with a "next_agent" field)
    response ={}
    response["next_agent"] = random.choice(["agent_2", END])
    # route to one of the agents or exit based on the LLM's decision
    # if the LLM returns "__end__", the graph will finish execution
    return Command(
        goto=response["next_agent"],
        update={"messages": state["messages"]},
    )

def agent_2(state: MessagesState) -> Command[Literal["agent_3", END]]:
    response ={}
    response["next_agent"] = random.choice(["agent_3", END])
    return Command(
        goto=response["next_agent"],
        update={"messages": state["messages"]},
    )

def agent_3(state: MessagesState) -> Command[Literal["agent_1", END]]:
    response ={}
    response["next_agent"] = random.choice(["agent_1", END])
    return Command(
        goto=response["next_agent"],
        update={"messages": state["messages"]},
    )

builder = StateGraph(MessagesState)
builder.add_node(agent_1)
builder.add_node(agent_2)
builder.add_node(agent_3)

builder.add_edge(START, "agent_1")
builder.add_edge("agent_1","agent_2")
builder.add_edge("agent_1","agent_3")
builder.add_edge("agent_1", END)

builder.add_edge("agent_2","agent_1")
builder.add_edge("agent_2","agent_3")
builder.add_edge("agent_2", END)

builder.add_edge("agent_3", "agent_1")
builder.add_edge("agent_3", END)
network = builder.compile()

In [96]:
network.invoke({"messages":"hello"},{"recursion_limit": 1000})

Task agent_1 with path ('__pregel_pull', 'agent_1') wrote to unknown channel branch:to:__end__, ignoring it.
Task agent_1 with path ('__pregel_pull', 'agent_1') wrote to unknown channel branch:to:__end__, ignoring it.
Task agent_3 with path ('__pregel_pull', 'agent_3') wrote to unknown channel branch:to:__end__, ignoring it.
Task agent_1 with path ('__pregel_pull', 'agent_1') wrote to unknown channel branch:to:__end__, ignoring it.
Task agent_2 with path ('__pregel_pull', 'agent_2') wrote to unknown channel branch:to:__end__, ignoring it.
Task agent_3 with path ('__pregel_pull', 'agent_3') wrote to unknown channel branch:to:__end__, ignoring it.
Task agent_1 with path ('__pregel_pull', 'agent_1') wrote to unknown channel branch:to:__end__, ignoring it.
Task agent_1 with path ('__pregel_pull', 'agent_1') wrote to unknown channel branch:to:__end__, ignoring it.
Task agent_2 with path ('__pregel_pull', 'agent_2') wrote to unknown channel branch:to:__end__, ignoring it.
Task agent_3 with p

GraphRecursionError: Recursion limit of 1000 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [97]:
# network 
# supervisor 
# supervisor(as tools)
# hierarchical

In [99]:
# custom multi-agent workflow
# explicit control flow - use normal edge
# dynamic control flow - use Command

In [ ]:
# communication and state management 

# handoff (using Command) vs tool calls

# message passing between agent (full message history vs only single final result)
# 1. sharing full thought process
# 2. shering only final results
# 3. indicating agent name in messages

# representing handoffs in message history 
# 1. add extra tool message to message list that indicate about handoff
# 2. remove all ai message with tool call

# state management for subagents - what if you want add tool calls message to messages list
# 1. store these messages in shared message list, but filter before passing it into subagent llm
# 2. store separate message list for each agent in subagent's graph agent 

# use different state schema
# 1. for communicate with each other subgraph and parent graph , use different state schema
# 2. use private input state schema


### Prebuild implmentation

In [1]:
# popular multi-agent architecture 
# supervisor: assign supervisor agent for control for multiple agents
# swarm: handoff control based on agent specification 

# handoffs:
# destination
# payloads

def transfer_to_bob():
    """Transfer to bob."""
    return Command(
        # name of the agent (node) to go to
        goto="bob",
        # data to send to the agent
        update={"messages": [...]},
        # indicate to LangGraph that we need to navigate to
        # agent node in a parent graph
        graph=Command.PARENT,
    )

### Custom implmentation

In [2]:
# use Command, for handoffs

# use Send, for send data for worker agents

In [ ]:
# multi-agent system 
from typing import Annotated
from langchain_core.messages import convert_to_messages
from langchain_core.tools import tool, InjectedToolCallId
from langgraph.prebuilt import create_react_agent, InjectedState
from langgraph.graph import StateGraph, START, MessagesState
from langgraph.types import Command

# We'll use `pretty_print_messages` helper to render the streamed agent outputs nicely

def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")


def create_handoff_tool(*, agent_name: str, description: str | None = None):
    name = f"transfer_to_{agent_name}"
    description = description or f"Transfer to {agent_name}"

    @tool(name, description=description)
    def handoff_tool(
        state: Annotated[MessagesState, InjectedState], 
        tool_call_id: Annotated[str, InjectedToolCallId],
    ) -> Command:
        tool_message = {
            "role": "tool",
            "content": f"Successfully transferred to {agent_name}",
            "name": name,
            "tool_call_id": tool_call_id,
        }
        return Command(  
            goto=agent_name,  
            update={"messages": state["messages"] + [tool_message]},  
            graph=Command.PARENT,  
        )
    return handoff_tool

# Handoffs
transfer_to_hotel_assistant = create_handoff_tool(
    agent_name="hotel_assistant",
    description="Transfer user to the hotel-booking assistant.",
)
transfer_to_flight_assistant = create_handoff_tool(
    agent_name="flight_assistant",
    description="Transfer user to the flight-booking assistant.",
)

# Simple agent tools
def book_hotel(hotel_name: str):
    """Book a hotel"""
    return f"Successfully booked a stay at {hotel_name}."

def book_flight(from_airport: str, to_airport: str):
    """Book a flight"""
    return f"Successfully booked a flight from {from_airport} to {to_airport}."


from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", api_key=GEMINI_API_KEY)

# Define agents
flight_assistant = create_react_agent(
    model=model,
    tools=[book_flight, transfer_to_hotel_assistant],
    prompt="You are a flight booking assistant",
    name="flight_assistant"
)
hotel_assistant = create_react_agent(
    model=model,
    tools=[book_hotel, transfer_to_flight_assistant],
    prompt="You are a hotel booking assistant",
    name="hotel_assistant"
)

# Define multi-agent graph
multi_agent_graph = (
    StateGraph(MessagesState)
    .add_node(flight_assistant)
    .add_node(hotel_assistant)
    .add_edge(START, "flight_assistant")
    .compile()
)

# Run the multi-agent graph
for chunk in multi_agent_graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "book a flight from BOS to JFK and a stay at McKittrick Hotel"
            }
        ]
    },
    subgraphs=True
):
    pretty_print_messages(chunk)

Update from subgraph flight_assistant:


	Update from node agent:


	================================== Ai Message ==================================
	Name: flight_assistant
	Tool Calls:
	  book_flight (5d9a78a9-ef1a-4f88-b3a0-8e082a308301)
	 Call ID: 5d9a78a9-ef1a-4f88-b3a0-8e082a308301
	  Args:
	    from_airport: BOS
	    to_airport: JFK


Update from subgraph flight_assistant:


	Update from node tools:


	================================= Tool Message =================================
	Name: book_flight
	
	Successfully booked a flight from BOS to JFK.


Update from subgraph flight_assistant:


	Update from node agent:


	================================== Ai Message ==================================
	Name: flight_assistant
	Tool Calls:
	  transfer_to_hotel_assistant (f1058e1b-eef3-4690-a913-e51151d00361)
	 Call ID: f1058e1b-eef3-4690-a913-e51151d00361
	  Args:


Update from subgraph hotel_assistant:


	Update from node agent:


	================================== Ai Message ======

In [ ]:
# multi-turn conversion

from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.prebuilt import create_react_agent, InjectedState
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import InMemorySaver


model = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", api_key=GEMINI_API_KEY)


class MultiAgentState(MessagesState):
    last_active_agent: str


# Define travel advisor tools and ReAct agent
travel_advisor_tools = [
    # get_travel_recommendations,
    create_handoff_tool(agent_name="hotel_advisor"),
]
travel_advisor = create_react_agent(
    model,
    travel_advisor_tools,
    prompt=(
        "You are a general travel expert that can recommend travel destinations (e.g. countries, cities, etc). "
        "If you need hotel recommendations, ask 'hotel_advisor' for help. "
        "You MUST include human-readable response before transferring to another agent."
    ),
)


def call_travel_advisor(
    state: MultiAgentState,
) -> Command[Literal["hotel_advisor", "human"]]:
    # You can also add additional logic like changing the input to the agent / output from the agent, etc.
    # NOTE: we're invoking the ReAct agent with the full history of messages in the state
    response = travel_advisor.invoke(state)
    update = {**response, "last_active_agent": "travel_advisor"}
    return Command(update=update, goto="human")


# Define hotel advisor tools and ReAct agent
hotel_advisor_tools = [
    get_hotel_recommendations,
    create_handoff_tool(agent_name="travel_advisor"),
]
hotel_advisor = create_react_agent(
    model,
    hotel_advisor_tools,
    prompt=(
        "You are a hotel expert that can provide hotel recommendations for a given destination. "
        "If you need help picking travel destinations, ask 'travel_advisor' for help."
        "You MUST include human-readable response before transferring to another agent."
    ),
)


def call_hotel_advisor(
    state: MultiAgentState,
) -> Command[Literal["travel_advisor", "human"]]:
    response = hotel_advisor.invoke(state)
    update = {**response, "last_active_agent": "hotel_advisor"}
    return Command(update=update, goto="human")


def human_node(
    state: MultiAgentState, config
) -> Command[Literal["hotel_advisor", "travel_advisor", "human"]]:
    """A node for collecting user input."""

    user_input = interrupt(value="Ready for user input.")
    active_agent = state["last_active_agent"]

    return Command(
        update={
            "messages": [
                {
                    "role": "human",
                    "content": user_input,
                }
            ]
        },
        goto=active_agent,
    )


builder = StateGraph(MultiAgentState)
builder.add_node("travel_advisor", call_travel_advisor)
builder.add_node("hotel_advisor", call_hotel_advisor)

# This adds a node to collect human input, which will route
# back to the active agent.
builder.add_node("human", human_node)

# We'll always start with a general travel advisor.
builder.add_edge(START, "travel_advisor")


checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)